<a href="https://colab.research.google.com/github/ezequielform/RepositorioFormacion/blob/main/Limpieza_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import pandas as pd
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [36]:

URL="/content/historical_automobile_sales.csv"
df = pd.read_csv(URL)
print(df.head())
print(df.dtypes)



        Date  Year Month  Recession  Consumer_Confidence  Seasonality_Weight  \
0  1/31/1980  1980   Jan          1               108.24                0.50   
1  2/29/1980  1980   Feb          1                98.75                0.75   
2  3/31/1980  1980   Mar          1               107.48                0.20   
3  4/30/1980  1980   Apr          1               115.01                1.00   
4  5/31/1980  1980   May          1                98.72                0.20   

       Price  Advertising_Expenditure  Competition     GDP  Growth_Rate  \
0  27483.571                     1558            7  60.223     0.010000   
1  24308.678                     3048            4  45.986    -0.309594   
2  28238.443                     3137            3  35.141    -0.308614   
3  32615.149                     1653            7  45.673     0.230596   
4  23829.233                     1319            4  52.997     0.138197   

   unemployment_rate  Automobile_Sales     Vehicle_Type        City 

In [37]:
#Cambio tipo de datos más apropiado
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Month'].astype('category')
df['Recession'] = df['Recession'].astype('category')
df['Competition'] = df['Competition'].astype('category')

print(df.dtypes)


Date                       datetime64[ns]
Year                                int64
Month                            category
Recession                        category
Consumer_Confidence               float64
Seasonality_Weight                float64
Price                             float64
Advertising_Expenditure             int64
Competition                      category
GDP                               float64
Growth_Rate                       float64
unemployment_rate                 float64
Automobile_Sales                  float64
Vehicle_Type                       object
City                               object
dtype: object


In [38]:
# Detectar valores nulos para cada columna y substituirlos por la media si es un valor numérico y por la moda si es un valor object o category.
# Si el número de valores nulos representa un % superior al 30%, eliminamos el valor nulo pero no el registro (que quede en blanco para que no interfiera el los cálculos estadísiticos)

def imputar_valores(df, umbral=0.30):
    df_copy = df.copy()
    num_filas = len(df_copy)

    for col in df_copy.columns:
        n_nulos = df_copy[col].isna().sum()
        pct_nulos = n_nulos / num_filas

        # Caso 1: numérico
        if pd.api.types.is_numeric_dtype(df_copy[col]):
            if pct_nulos < umbral:
                # Reemplazar por la media
                media = df_copy[col].mean()
                df_copy[col].fillna(media, inplace=True)
            # Si supera el umbral → no se toca (se deja NaN)

        # Caso 2: categórico u objeto
        elif pd.api.types.is_object_dtype(df_copy[col]) or pd.api.types.is_categorical_dtype(df_copy[col]):
            if pct_nulos < umbral:
                # Reemplazar por la moda
                moda = df_copy[col].mode()
                if len(moda) > 0: # por si el resultado vacío es el  más repetido
                    df_copy[col].fillna(moda.iloc[0], inplace=True)
            # Si supera el umbral → no se toca

    return df_copy


# Aplicarlo al dataframe
df_limpio = imputar_valores(df)



print("\nNulos por columna DESPUÉS de imputar:\n")
print(df_limpio.isna().sum())





Nulos por columna DESPUÉS de imputar:

Date                       0
Year                       0
Month                      0
Recession                  0
Consumer_Confidence        0
Seasonality_Weight         0
Price                      0
Advertising_Expenditure    0
Competition                0
GDP                        0
Growth_Rate                0
unemployment_rate          0
Automobile_Sales           0
Vehicle_Type               0
City                       0
dtype: int64


/tmp/ipython-input-941054096.py:21: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_object_dtype(df_copy[col]) or pd.api.types.is_categorical_dtype(df_copy[col]):
/tmp/ipython-input-941054096.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy[col].fillna(media, inplace=True)
/tmp/ipython-input-941054096.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through ch

In [40]:
#Reemplazar atípicos por Nan
def eliminar_outliers_iqr(df):
    df_clean = df.copy()
    cols_numericas = df_clean.select_dtypes(include=['number']).columns

    print("Reemplazando outliers en columnas numéricas por NaN...\n")

    for col in cols_numericas:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lim_inf = Q1 - 1.5 * IQR
        lim_sup = Q3 + 1.5 * IQR

        # reemplaza solo los valores atípicos en la columna actual por NaN
        n_outliers = ((df_clean[col] < lim_inf) | (df_clean[col] > lim_sup)).sum()
        df_clean.loc[(df_clean[col] < lim_inf) | (df_clean[col] > lim_sup), col] = pd.NA

        print(f"{col}: reemplazados {n_outliers} valores atípicos")

    print("\nTotal de filas después de limpiar:", df_clean.shape[0])

    return df_clean

# Aplicarlo al dataframe
df_sin_outliers = eliminar_outliers_iqr(df_limpio)



Reemplazando outliers en columnas numéricas por NaN...

Year: reemplazados 0 valores atípicos
Consumer_Confidence: reemplazados 2 valores atípicos
Seasonality_Weight: reemplazados 0 valores atípicos
Price: reemplazados 4 valores atípicos
Advertising_Expenditure: reemplazados 0 valores atípicos
GDP: reemplazados 0 valores atípicos
Growth_Rate: reemplazados 28 valores atípicos
unemployment_rate: reemplazados 33 valores atípicos
Automobile_Sales: reemplazados 1 valores atípicos

Total de filas después de limpiar: 528
